In [3]:
import pandas as pd
import numpy as np


In [6]:
df = pd.read_feather('../data/train_dataset.fth')
df.head()


,index,id,cell_id,cell_type,source,rank,pct_rank,ancestor_id,parent_id
0,0,00001756c60be8,1862f0a6,code,# This Python 3 environment comes with many he...,0,0.000000,945aea18,None
1,1,00001756c60be8,448eb224,markdown,**Импортируем необходимые для работы функции и...,1,0.017241,945aea18,None
2,2,00001756c60be8,2a9e43d6,code,import numpy as np\nimport pandas as pd\nimpor...,2,0.034483,945aea18,None
3,3,00001756c60be8,7e2f170a,markdown,**Подключаем предупреждения**,3,0.051724,945aea18,None
4,4,00001756c60be8,038b763d,code,import warnings\nwarnings.filterwarnings('igno...,4,0.068966,945aea18,None


In [26]:
plot_functions = {
    'histplot', 'kdeplot', 'displot', 
    'displot', 'scatterplot', 'relplot', 
    'jointplot', 'plot', 'show',
    'pairplot', 'countplot', 'jointplot'
    'relplot', 'lmplot', 'catplot',
    'scatter', 'hist'
}

In [133]:
from tqdm import tqdm
import tokenize
import io

def get_functions_and_variables(source):
    try:
        code_text = tokenize.generate_tokens(io.StringIO(source).readline)
        fv_list = [tok.string for tok in code_text if tok.type==1]
    except (tokenize.TokenError, IndentationError) as e:
        return [], []
        
    fv_plot = [s for s in fv_list if any([s in p for p in plot_functions])]
    return fv_list, fv_plot

    
def generate_features(df):
    features = dict()
    
    cell_counts = df.groupby('id').cell_type.value_counts().unstack()
    cell_counts['ratio'] = cell_counts['markdown'] / cell_counts['code']
    cell_counts['total'] = (cell_counts['markdown'] + cell_counts['code']).astype(int)
    mean_ratios_by_count = cell_counts.groupby('total').ratio.mean()    
    
    for idx, sub_df in tqdm(df.groupby("id")):
        features[idx] = dict()
        total_md = cell_counts.loc[idx].markdown
        total_code = cell_counts.loc[idx].code

        code_sub_df = sub_df[sub_df.cell_type == "code"]
        source_code = '\n'.join(code_sub_df.source)
        funcs_and_vars, plot_funcs = get_functions_and_variables(source_code)
        defined_functions = re.findall('def (.*)\(', source_code)
        
        features[idx] = {
            "total_code": total_code, 
            "total_md": total_md,
            
            "defined_functions": defined_functions,
            "normalized_defined_functions": len(plot_funcs) / total_code,
            "normalized_plot_functions": len(defined_functions) / total_code,
            
            "mean_ratio_in_same_notebooks": mean_ratios_by_count.loc[total_md + total_code],
            "normalized_sloc": len(source_code.splitlines()) / total_code,
        }
        
    return features


In [ ]:
ft = generate_features(df.iloc[:100000])


  8%|▊         | 166/2158 [00:01<00:13, 152.71it/s]

In [135]:
ft

{'00001756c60be8': {'total_code': 30.0,
  'total_md': 28.0,
  'defined_functions': ['evaluate_preds',
   '__init__',
   'fit',
   'transform',
   'features'],
  'normalized_defined_functions': 0.5333333333333333,
  'normalized_plot_functions': 0.16666666666666666,
  'mean_ratio_in_same_notebooks': 0.9333333333333333,
  'normalized_sloc': 9.0},
 '00015c83e2717b': {'total_code': 72.0,
  'total_md': 21.0,
  'defined_functions': ['mask2rgba',
   'name_and_mask',
   'show_mask_image',
   'mask2rle',
   'rle2mask',
   'vflip',
   'hflip',
   'random_flip',
   'transpose',
   'rot90',
   'rotate',
   'shift_scale_rotate',
   'center_crop',
   'clip',
   'clipped',
   'wrapped_function',
   'shift_hsv',
   'shift_rgb',
   'clahe(img, clipLimit=2.0, tileGridSize=',
   'blur',
   'median_blur',
   'motion_blur',
   'random_polosa',
   'distort1',
   'distort2',
   'elastic_transform_fast',
   'remap_color',
   'get_lut',
   'invert',
   'channel_shuffle',
   'gauss_noise',
   'salt_pepper_noise'